In [ ]:
####################################################################################
# create and save for samebasin intake files
#cityrivnum = f'{h08dir}/global_city/dat/riv_num_/city_basins.json'
#city_basins_list
####################################################################################
    
import os
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import pandas as pd
import matplotlib.colors as mcolors

####################################################################################
city_len = 1860
distance = 100
ex_flg = False
####################################################################################

# paths
h08dir = '/mnt/c/Users/tsimk/Downloads/dotfiles/h08'
pop_path = f'{h08dir}/global_city/dat/pop_tot_/GPW4ag__20100000.gl5'
riv_path = f'{h08dir}/wsi/dat/riv_out_/W5E5LR__00000000.gl5'
rivnum_path = f'{h08dir}/global_city/dat/riv_num_/rivnum.CAMA.gl5'
rivnxl_path = f"{h08dir}/global_city/dat/riv_nxl_/rivnxl.CAMA.gl5"
msk_dir = f'{h08dir}/global_city/dat/vld_cty_'
cnt_dir = f'{h08dir}/global_city/dat/cty_cnt_/gpw4'
prf_dir = f'{h08dir}/global_city/dat/cty_prf_'

# intake
int_dir = f'{h08dir}/global_city/dat/cty_int_/{distance}km_samebasin'

# rivpath
rivnxl_gl5 = np.fromfile(rivnxl_path, 'float32').reshape(2160, 4320)
pop = np.fromfile(pop_path, dtype='float32').reshape(2160, 4320)
rivout = np.fromfile(riv_path, dtype='float32').reshape(2160, 4320)
rivnum_gl5 = np.fromfile(rivnum_path, dtype='float32').reshape(2160, 4320)

# lonlat
lonlat_path = f'{h08dir}/global_city/dat/cty_lst_/gpw4/WUP2018_300k_2010.txt'
with open(lonlat_path, "r") as input_file:
    lines = input_file.readlines()

# identifying same rivnum cities into list?
city_basins = {}
for i in range(city_len):
    city_num = i+1
    ctymsk_path = f'{msk_dir}/city_{city_num:08}.gl5'
    prf_path = f'{prf_dir}/vld_cty_/city_{city_num:08}.gl5'
    int_path = f'{int_dir}/city_{city_num:08}.gl5'

    if not os.path.exists(prf_path):
        print(f'{city_num} is invalid prf')

    else:
        ctymsk = np.fromfile(ctymsk_path, dtype='float32').reshape(2160, 4320)
        prf = np.fromfile(prf_path, dtype='float32').reshape(2160, 4320)
        intake = np.fromfile(int_path, dtype='float32').reshape(2160, 4320)
        rivnum_mask = np.ma.masked_where((intake+prf) == 0, rivnum_gl5)
        rivnum_lst = np.unique(rivnum_mask)[:-1]
        city_basins[city_num] = rivnum_lst.filled(1e20)
        
        print(f'{city_num} done')

print(city_basins)

"""
basin_to_cities = {}
for city, basins in city_basins.items():
    for rivnum in basins:
        if rivnum not in basin_to_cities:
            basin_to_cities[rivnum] = []
        basin_to_cities[rivnum].append(city)

# 集計結果を出力
for rivnum, cities in basin_to_cities.items():
    if len(cities) > 1:  # 複数の都市に共有されている流域番号のみ表示
        print(f"rivnum {rivnum}: {cities}")

import json
city_basins_list = {city: basins.tolist() for city, basins in city_basins.items()}
cityrivnum = f'{h08dir}/global_city/dat/riv_num_/city_basins_{distance}km.json'
with open(cityrivnum, 'w') as json_file:
    json.dump(city_basins_list, json_file)
print(f'{cityrivnum} saved')
"""

1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done
10 done
11 done
12 done
13 done
14 done
15 done
16 done
17 done
18 done
19 done
20 done
21 done
22 done
23 done


In [3]:
####################################################################################
# create and save below files
#basin_cities_path = f'{h08dir}/global_city/dat/riv_num_/basin_to_cities.json'
#new_basin_to_cities
####################################################################################

import os
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import pandas as pd
import matplotlib.colors as mcolors

def l_coordinate_to_tuple(lcoordinate, a=2160, b=4320):
    lat_l = ((lcoordinate - 1) // b)
    lon_l = (lcoordinate) % b - 1
    return (lat_l, lon_l)

def nxtl2nxtxy(rgnfile, upperindex, leftindex):
    vfunc = np.vectorize(l_coordinate_to_tuple, otypes=[tuple])
    riv_nxtxy = np.empty(rgnfile.shape, dtype=tuple)
    mask = ~np.isnan(rgnfile)
    riv_nxtxy[mask] = vfunc(rgnfile[mask])
    riv_nxtxy_shape = (riv_nxtxy.shape[0], riv_nxtxy.shape[1], 2)
    
    riv_nxtxy_lst = []
    for row in riv_nxtxy:
        for y, x in row:
            modified_y = y - upperindex
            modified_x = x - leftindex
            riv_nxtxy_lst.append((modified_y, modified_x))

    riv_nxtxy_cropped = np.array(riv_nxtxy_lst).reshape(riv_nxtxy_shape)
    riv_nxtxy_cropped = riv_nxtxy_cropped.astype(int)
    return riv_nxtxy_cropped
    
################################################################################
# load files
################################################################################
city_len = 1860
distance = 100
ex_flg = False
h08dir = '/mnt/c/Users/tsimk/Downloads/dotfiles/h08'
rivnxl_path = f"{h08dir}/global_city/dat/riv_nxl_/rivnxl.CAMA.gl5"

# rivpath
rivnxl_gl5 = np.fromfile(rivnxl_path, 'float32').reshape(2160, 4320)
################################################################################

# rivnxl in xy coord
riv_nxlonlat_cropped = nxtl2nxtxy(rivnxl_gl5, 0, 0)

# basin at each city
cityrivnum = f'{h08dir}/global_city/dat/riv_num_/city_basins_{distance}km.json'
with open(cityrivnum, 'r') as json_file:
    city_basins = json.load(json_file)
print(len(city_basins.keys()))

# cities at same basin
basin_to_cities = {}
for city, basins in city_basins.items():
    for rivnum in basins:
        if rivnum not in basin_to_cities:
            basin_to_cities[rivnum] = []
        basin_to_cities[rivnum].append(city)
        
for rivnum, cities in basin_to_cities.items():
    if len(cities) > 1:  # 複数の都市に共有されている流域番号のみ表示
        print(f"rivnum {rivnum}: {cities}")
        
new_basin_to_cities = {rivnum: cities for rivnum, cities in basin_to_cities.items() if len(cities) > 1}
basin_cities_path = f'{h08dir}/global_city/dat/riv_num_/basin_to_cities_{distance}km.json'

"""
with open(basin_cities_path, 'w') as json_file:
    json.dump(new_basin_to_cities, json_file)
    print(f'{basin_cities_path} saved')
"""

1553
rivnum 848.0: ['1', '936', '1283', '1400']
rivnum 14.0: ['2', '11', '126', '130', '133', '188', '197', '217', '243', '319', '321', '363', '398', '406', '455', '463', '473', '513', '526', '588', '667', '669', '673', '677', '797', '874', '883', '903', '941', '983', '1012', '1018', '1067', '1089', '1181', '1216', '1253', '1288', '1298', '1337', '1365', '1371', '1381', '1422', '1575', '1578', '1620', '1652', '1656', '1668', '1697', '1712', '1718', '1824']
rivnum 480.0: ['3', '52', '85', '144', '160', '275', '612', '618', '873', '967', '1200', '1299', '1376', '1687']
rivnum 5438.0: ['3', '85']
rivnum 9439.0: ['3', '1212']
rivnum 112.0: ['4', '74', '209', '269', '397', '431', '483', '557', '779', '1104']
rivnum 164.0: ['4', '537', '929', '933', '1569']
rivnum 1366.0: ['4', '143', '487', '933', '953', '1094']
rivnum 5.0: ['5', '94', '119', '140', '155', '196', '257', '343', '550', '597', '629', '689', '767', '789', '803', '836', '837', '866', '939', '1125', '1235', '1327', '1329', '1361'

In [3]:
####################################################################################
# テストする場合はここから始めてください
# edge experiment, how coords_a looks like
####################################################################################
key_index=1
distance=100

import json
import os
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import pandas as pd
import matplotlib.colors as mcolors

h08dir = '/mnt/c/Users/tsimk/Downloads/dotfiles/h08'
basin_cities_path = f'{h08dir}/global_city/dat/riv_num_/basin_to_cities_{distance}km.json'
with open(basin_cities_path, 'r') as json_file:
    new_basin_to_cities = json.load(json_file)

# get uid and city list
keys_list = list(new_basin_to_cities.keys())
uid = keys_list[key_index]
print(f'uid: {uid}')
rivnum_list = new_basin_to_cities[uid]
rivnum_list = [int(i) for i in rivnum_list]

# remove overlap
h08dir = '/mnt/c/Users/tsimk/Downloads/dotfiles/h08'
overlap_path = f'{h08dir}/global_city/dat/cty_lst_/gpw4/overlap_hidden_only.txt'
with open(overlap_path, 'r') as f:
    numbers = [int(line.strip()) for line in f]
rivnum_list_removed = [num for num in rivnum_list if num not in numbers]
rivnum_list_removed = [int(i) for i in rivnum_list_removed]
print('cities in uid')
print(rivnum_list_removed)

# coord of purficication
coords_a = []
for city_num in rivnum_list_removed:
    # rivnum
    rivnum_path = f'{h08dir}/global_city/dat/riv_num_/rivnum.CAMA.gl5'
    rivnum_gl5 = np.fromfile(rivnum_path, dtype='float32').reshape(2160, 4320)

    # prf
    prf_dir = f'{h08dir}/global_city/dat/cty_prf_'
    prf_path = f'{prf_dir}/vld_cty_/city_{city_num:08}.gl5'
    prf = np.fromfile(prf_path, dtype='float32').reshape(2160, 4320)

    # int
    int_dir = f'{h08dir}/global_city/dat/cty_int_/{distance}km_samebasin'
    int_path = f'{int_dir}/city_{city_num:08}.gl5'
    intake = np.fromfile(int_path, dtype='float32').reshape(2160, 4320)
    
    # 1. prfのみが対象流域内
    # 2. intのみが対象流域内
    # 3. prfとintどちらもが対象領域内
    uid = int(float(uid))
    prf_coord = np.where((prf == 1) & (rivnum_gl5 == uid))
    int_coord = np.where((intake==1) & (rivnum_gl5 == uid))
    
    if prf_coord[0].size == 0:
        continue
    else:
        coords_a.append([prf_coord, city_num])
        
    if int_coord[0].size == 0:
        continue
    else:
        coords_a.append([int_coord, city_num])
        #print(coords_a[0][0][0]) #[737]
        #print(coords_a[0][0][1]) #[3082]
        #print(coords_a[0][1]) #2
        
print('coords_a')
print(coords_a)
print(coords_a[0])
print(coords_a[0][0][0][0])
print(coords_a[0][0][1][0])
print(coords_a[0][1])

uid: 14.0
cities in uid
[2, 11, 126, 130, 133, 188, 197, 217, 243, 319, 321, 363, 398, 406, 455, 463, 473, 513, 526, 588, 667, 669, 673, 677, 797, 874, 883, 903, 941, 983, 1012, 1018, 1067, 1089, 1181, 1216, 1253, 1288, 1298, 1337, 1365, 1371, 1422, 1578, 1620, 1652, 1656, 1697, 1712, 1718, 1824]
coords_a
[[(array([737]), array([3082])), 2], [(array([732]), array([3086])), 2], [(array([790]), array([3245])), 11], [(array([786]), array([3237])), 11], [(array([755]), array([3068])), 126], [(array([761]), array([3123])), 130], [(array([756]), array([3130])), 130], [(array([757]), array([3130])), 133], [(array([762]), array([3124])), 133], [(array([808]), array([3070])), 188], [(array([809]), array([3069])), 188], [(array([772]), array([3181])), 197], [(array([770]), array([3182])), 197], [(array([801]), array([3087])), 217], [(array([798]), array([3098])), 217], [(array([753]), array([3095])), 243], [(array([757]), array([3095])), 243], [(array([776]), array([3152])), 319], [(array([786])

In [8]:
####################################################################################
# edgesは2都市間のupstreamとdownstreamの関係をすべて保存したリスト
####################################################################################

import json

h08dir = '/mnt/c/Users/tsimk/Downloads/dotfiles/h08'
basin_cities_path = f'{h08dir}/global_city/dat/riv_num_/basin_to_cities_{distance}km.json'
with open(basin_cities_path, 'r') as json_file:
    new_basin_to_cities = json.load(json_file)
    
def l_coordinate_to_tuple(lcoordinate, a=2160, b=4320):
    lat_l = ((lcoordinate - 1) // b)
    lon_l = (lcoordinate) % b - 1
    return (lat_l, lon_l)

def nxtl2nxtxy(rgnfile, upperindex, leftindex):
    vfunc = np.vectorize(l_coordinate_to_tuple, otypes=[tuple])
    riv_nxtxy = np.empty(rgnfile.shape, dtype=tuple)
    mask = ~np.isnan(rgnfile)
    riv_nxtxy[mask] = vfunc(rgnfile[mask])
    riv_nxtxy_shape = (riv_nxtxy.shape[0], riv_nxtxy.shape[1], 2)
    
    riv_nxtxy_lst = []
    for row in riv_nxtxy:
        for y, x in row:
            modified_y = y - upperindex
            modified_x = x - leftindex
            riv_nxtxy_lst.append((modified_y, modified_x))

    riv_nxtxy_cropped = np.array(riv_nxtxy_lst).reshape(riv_nxtxy_shape)
    riv_nxtxy_cropped = riv_nxtxy_cropped.astype(int)
    return riv_nxtxy_cropped
    
rivnxl_path = f"{h08dir}/global_city/dat/riv_nxl_/rivnxl.CAMA.gl5"
rivnxl_gl5 = np.fromfile(rivnxl_path, 'float32').reshape(2160, 4320)
riv_nxlonlat_cropped = nxtl2nxtxy(rivnxl_gl5, 0, 0)

def updown(new_basin_to_cities, key_index=0, distance=100):
    # get uid and city list
    keys_list = list(new_basin_to_cities.keys())
    uid = keys_list[key_index]
    print(f'uid: {uid}')
    rivnum_list = new_basin_to_cities[uid]
    rivnum_list = [int(i) for i in rivnum_list]
    
    # remove overlap
    h08dir = '/mnt/c/Users/tsimk/Downloads/dotfiles/h08'
    overlap_path = f'{h08dir}/global_city/dat/cty_lst_/gpw4/overlap_hidden_only.txt'
    with open(overlap_path, 'r') as f:
        numbers = [int(line.strip()) for line in f]
    rivnum_list_removed = [num for num in rivnum_list if num not in numbers]
    rivnum_list_removed = [int(i) for i in rivnum_list_removed]
    print('cities in uid')
    print(rivnum_list_removed)
    
    # coord of purficication
    coords_a = []
    for city_num in rivnum_list_removed:
        # rivnum
        rivnum_path = f'{h08dir}/global_city/dat/riv_num_/rivnum.CAMA.gl5'
        rivnum_gl5 = np.fromfile(rivnum_path, dtype='float32').reshape(2160, 4320)

        # prf
        prf_dir = f'{h08dir}/global_city/dat/cty_prf_'
        prf_path = f'{prf_dir}/vld_cty_/city_{city_num:08}.gl5'
        prf = np.fromfile(prf_path, dtype='float32').reshape(2160, 4320)

        # int
        int_dir = f'{h08dir}/global_city/dat/cty_int_/{distance}km_samebasin'
        int_path = f'{int_dir}/city_{city_num:08}.gl5'
        intake = np.fromfile(int_path, dtype='float32').reshape(2160, 4320)
        
        # 1. prfのみが対象流域内
        # 2. intのみが対象流域内
        # 3. prfとintどちらもが対象領域内
        uid = int(float(uid))
        prf_coord = np.where((prf == 1) & (rivnum_gl5 == uid))
        int_coord = np.where((intake==1) & (rivnum_gl5 == uid))
        
        if prf_coord[0].size == 0:
            continue
        else:
            coords_a.append([prf_coord, city_num])
            
        if int_coord[0].size == 0:
            continue
        else:
            coords_a.append([int_coord, city_num])
            #print(coords_a[0][0][0]) #[737]
            #print(coords_a[0][0][1]) #[3082]
            #print(coords_a[0][1]) #2
            
    print('coords_a')
    print(coords_a)
    
    # down
    edges = []
    riv_path_array = np.zeros((2160, 4320))
    for idx in range(len(coords_a)):
        visited_coords = set()
        riv_path_array[coords_a[idx][0][0][0], coords_a[idx][0][1][0]] = idx

        # coordinates of prf and intake
        coords_b = coords_a.copy()
        coords_b.pop(idx) 
        
        if len(coords_a) > 0:
            target_coord = (coords_a[idx][0][0][0], coords_a[idx][0][1][0])
            
            while True:
                if target_coord in visited_coords:
                    break
                visited_coords.add(target_coord)
                
                next_coord = riv_nxlonlat_cropped[target_coord[0], target_coord[1]]
                if next_coord.size == 0 or next_coord.shape != (2,):
                    break
                
                next_coord = (next_coord[0], next_coord[1])
                riv_path_array[next_coord[0], next_coord[1]] = idx
                target_coord = next_coord
        
        for cind, coord in enumerate(coords_b):
            if coord[0][0].size == 0:
                continue
            else:
                standard_coord = (coord[0][0][0], coord[0][1][0])
                if standard_coord in visited_coords:
                    edges.append((coords_a[idx][1], coord[1]))

    # edgesは2都市間のupstreamとdownstreamの関係をすべて保存したリスト
    return edges, riv_path_array, coords_a, rivnum_list_removed
    
# unique_cties_basin: uniqueなすべての都市
# uup: 各都市の上流にあるuniqueな都市
# key_index = city_num like index
edges, riv_path_array, coords_a, rivnum_list_removed = updown(new_basin_to_cities, key_index=1)
unique_cities_basin = np.unique(edges)

uid: 14.0
cities in uid
[2, 11, 126, 130, 133, 188, 197, 217, 243, 319, 321, 363, 398, 406, 455, 463, 473, 513, 526, 588, 667, 669, 673, 677, 797, 874, 883, 903, 941, 983, 1012, 1018, 1067, 1089, 1181, 1216, 1253, 1288, 1298, 1337, 1365, 1371, 1422, 1578, 1620, 1652, 1656, 1697, 1712, 1718, 1824]
coords_a
[[(array([737]), array([3082])), 2], [(array([732]), array([3086])), 2], [(array([790]), array([3245])), 11], [(array([786]), array([3237])), 11], [(array([755]), array([3068])), 126], [(array([761]), array([3123])), 130], [(array([756]), array([3130])), 130], [(array([757]), array([3130])), 133], [(array([762]), array([3124])), 133], [(array([808]), array([3070])), 188], [(array([809]), array([3069])), 188], [(array([772]), array([3181])), 197], [(array([770]), array([3182])), 197], [(array([801]), array([3087])), 217], [(array([798]), array([3098])), 217], [(array([753]), array([3095])), 243], [(array([757]), array([3095])), 243], [(array([776]), array([3152])), 319], [(array([786])

In [9]:
####################################################################################
# すべての流域を含むresult_dictを作成
# 同じ流域に含まれる各都市のwater availability, upstream citiesを保存している．cities_with_upstreamが存在する場合，flag=Trueとなる
####################################################################################

def find_upstream_cities(edges, city_number):
    cities_with_upstream = []
    cities_without_upstream = []
    upstream_cities = [edge[0] for edge in edges if edge[1] == city_number]
    uup = np.unique(upstream_cities)
    
    for i in uup:
        if any(edge[1] == i for edge in edges):
            # 上流にさらに都市が存在する場合
            cities_with_upstream.append(i)
        else:
            # 上流にこれ以上都市が存在しない場合
            cities_without_upstream.append(i)
    
    return cities_with_upstream, cities_without_upstream, uup
    
# save file
result_dict = {}

# job
for city_number in unique_cities_basin:
    cities_with_upstream, cities_without_upstream, uup = find_upstream_cities(edges, city_number)
    if cities_with_upstream:
        result_dict[city_number] = [1, True, uup]
    elif not cities_with_upstream and cities_without_upstream:
        result_dict[city_number] = [1, False, uup]
    else:
        result_dict[city_number] = [1, 'Done', uup]

# 同じ流域に含まれる各都市のwater availability, upstream citiesを保存している．cities_with_upstreamが存在する場合，flag=Trueとなる
for rivnum_key in result_dict:
    value = result_dict[rivnum_key]
    print(rivnum_key, value)

2 [1, True, array([ 667, 1089, 1253, 1288, 1422, 1620])]
11 [1, True, array([ 473,  673, 1181, 1365])]
126 [1, 'Done', array([], dtype=float64)]
130 [1, True, array([ 321,  463,  513,  526,  667,  677,  797,  983, 1253, 1371, 1656,
       1824])]
133 [1, True, array([ 130,  321,  463,  513,  526,  667,  677,  797,  983, 1253, 1371,
       1656, 1824])]
188 [1, 'Done', array([], dtype=float64)]
197 [1, True, array([   2,  126,  130,  133,  188,  217,  243,  319,  321,  363,  406,
        455,  463,  513,  526,  667,  669,  677,  797,  883,  941,  983,
       1012, 1018, 1067, 1089, 1253, 1288, 1298, 1337, 1371, 1422, 1620,
       1656, 1697, 1712, 1718, 1824])]
217 [1, 'Done', array([], dtype=float64)]
243 [1, True, array([   2,  667,  677,  983, 1067, 1089, 1253, 1288, 1422, 1620])]
319 [1, False, array([1697])]
321 [1, True, array([ 667,  677,  983, 1253])]
363 [1, True, array([   2,  126,  130,  133,  188,  217,  243,  321,  406,  455,  463,
        513,  526,  667,  677,  797,  883,

In [12]:
######################################################################
# JOB START 
#(初めてでなければここまでの計算を飛ばして，ここからスタートしてもよい)
######################################################################

import os
import json
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import pandas as pd
import matplotlib.colors as mcolors

def l_coordinate_to_tuple(lcoordinate, a=2160, b=4320):
    lat_l = ((lcoordinate - 1) // b)
    lon_l = (lcoordinate) % b - 1
    return (lat_l, lon_l)

def nxtl2nxtxy(rgnfile, upperindex, leftindex):
    vfunc = np.vectorize(l_coordinate_to_tuple, otypes=[tuple])
    riv_nxtxy = np.empty(rgnfile.shape, dtype=tuple)
    mask = ~np.isnan(rgnfile)
    riv_nxtxy[mask] = vfunc(rgnfile[mask])
    riv_nxtxy_shape = (riv_nxtxy.shape[0], riv_nxtxy.shape[1], 2)
    
    riv_nxtxy_lst = []
    for row in riv_nxtxy:
        for y, x in row:
            modified_y = y - upperindex
            modified_x = x - leftindex
            riv_nxtxy_lst.append((modified_y, modified_x))

    riv_nxtxy_cropped = np.array(riv_nxtxy_lst).reshape(riv_nxtxy_shape)
    riv_nxtxy_cropped = riv_nxtxy_cropped.astype(int)
    return riv_nxtxy_cropped

# init
h08dir = '/mnt/c/Users/tsimk/Downloads/dotfiles/h08'
pop_path = f'{h08dir}/global_city/dat/pop_tot_/GPW4ag__20100000.gl5'
rivnxl_path = f"{h08dir}/global_city/dat/riv_nxl_/rivnxl.CAMA.gl5"
rivout_path = f'{h08dir}/wsi/dat/riv_out_/W5E5LR__00000000.gl5'
rivnum_path = f'{h08dir}/global_city/dat/riv_num_/rivnum.CAMA.gl5'
rivara_path = f'{h08dir}/global_city/dat/riv_ara_/rivara.CAMA.gl5'
msk_dir = f'{h08dir}/global_city/dat/vld_cty_'
prf_dir = f'{h08dir}/global_city/dat/cty_prf_'

pop_gl5 = np.fromfile(pop_path, dtype='float32').reshape(2160, 4320)
rivout_gl5 = np.fromfile(rivout_path, dtype='float32').reshape(2160, 4320)
rivnum_gl5 = np.fromfile(rivnum_path, dtype='float32').reshape(2160, 4320)
rivara_gl5 = np.fromfile(rivara_path, dtype='float32').reshape(2160, 4320)
rivnxl_gl5 = np.fromfile(rivnxl_path, 'float32').reshape(2160, 4320)
riv_nxlonlat_cropped = nxtl2nxtxy(rivnxl_gl5, 0, 0)

def load_json(distance=100):
    basin_cities_path = f'{h08dir}/global_city/dat/riv_num_/basin_to_cities_{distance}km.json'
    with open(basin_cities_path, 'r') as json_file:
        basin_to_cities = json.load(json_file)
    new_basin_to_cities = {rivnum: cities for rivnum, cities in basin_to_cities.items() if len(cities) > 1}
    return new_basin_to_cities

def updown(new_basin_to_cities, key_index=0, distance=100):
    # get uid and city list
    keys_list = list(new_basin_to_cities.keys())
    uid = keys_list[key_index]
    print(f'uid: {uid}')
    rivnum_list = new_basin_to_cities[uid]
    rivnum_list = [int(i) for i in rivnum_list]
    
    # remove overlap
    h08dir = '/mnt/c/Users/tsimk/Downloads/dotfiles/h08'
    overlap_path = f'{h08dir}/global_city/dat/cty_lst_/gpw4/overlap_hidden_only.txt'
    with open(overlap_path, 'r') as f:
        numbers = [int(line.strip()) for line in f]
    rivnum_list_removed = [num for num in rivnum_list if num not in numbers]
    rivnum_list_removed = [int(i) for i in rivnum_list_removed]
    print('cities in uid')
    print(rivnum_list_removed)
    
    # coord of purficication
    coords_a = []
    for city_num in rivnum_list_removed:
        # rivnum
        rivnum_path = f'{h08dir}/global_city/dat/riv_num_/rivnum.CAMA.gl5'
        rivnum_gl5 = np.fromfile(rivnum_path, dtype='float32').reshape(2160, 4320)

        # prf
        prf_dir = f'{h08dir}/global_city/dat/cty_prf_'
        prf_path = f'{prf_dir}/vld_cty_/city_{city_num:08}.gl5'
        prf = np.fromfile(prf_path, dtype='float32').reshape(2160, 4320)

        # int
        int_dir = f'{h08dir}/global_city/dat/cty_int_/{distance}km_samebasin'
        int_path = f'{int_dir}/city_{city_num:08}.gl5'
        intake = np.fromfile(int_path, dtype='float32').reshape(2160, 4320)
        # don't think about IBT updown at current stage
        intake = np.zeros((2160, 4320))
        
        uid = int(float(uid))
        prf_coord = np.where((prf == 1) & (rivnum_gl5 == uid))
        int_coord = np.where((intake==1) & (rivnum_gl5 == uid))
        
        if prf_coord[0].size == 0:
            continue
        else:
            coords_a.append([prf_coord, city_num])
            
        if int_coord[0].size == 0:
            continue
        else:
            coords_a.append([int_coord, city_num])

    # down
    edges = []
    riv_path_array = np.zeros((2160, 4320))
    for idx in range(len(coords_a)):
        visited_coords = set()
        riv_path_array[coords_a[idx][0][0][0], coords_a[idx][0][1][0]] = idx

        # coordinates of prf and intake
        coords_b = coords_a.copy()
        coords_b.pop(idx) 
        
        if len(coords_a) > 0:
            target_coord = (coords_a[idx][0][0][0], coords_a[idx][0][1][0])
            
            while True:
                if target_coord in visited_coords:
                    break
                visited_coords.add(target_coord)
                
                next_coord = riv_nxlonlat_cropped[target_coord[0], target_coord[1]]
                if next_coord.size == 0 or next_coord.shape != (2,):
                    break
                
                next_coord = (next_coord[0], next_coord[1])
                riv_path_array[next_coord[0], next_coord[1]] = idx
                target_coord = next_coord
        
        for cind, coord in enumerate(coords_b):
            if coord[0][0].size == 0:
                continue
            else:
                standard_coord = (coord[0][0][0], coord[0][1][0])
                if standard_coord in visited_coords:
                    edges.append((coords_a[idx][1], coord[1]))

    # edgesは2都市間のupstreamとdownstreamの関係をすべて保存したリスト
    return edges, riv_path_array, coords_a, rivnum_list_removed
    
def find_upstream_cities(edges, city_number):
    cities_with_upstream = []
    cities_without_upstream = []
    upstream_cities = [edge[0] for edge in edges if edge[1] == city_number]
    uup = np.unique(upstream_cities)
    
    for i in uup:
        if any(edge[1] == i for edge in edges):
            # 上流にさらに都市が存在する場合
            cities_with_upstream.append(i)
        else:
            # 上流にこれ以上都市が存在しない場合
            cities_without_upstream.append(i)
    
    return cities_with_upstream, cities_without_upstream, uup
    
mcy_pop_water_path = '/mnt/c/Users/tsimk/Downloads/dotfiles/h08/global_city/dat/cty_lst_/gpw4/mcy_pop_water_12region.txt'
column_names = ['index', 'flag', 'rate', 'wup', 'pop', 'ava', 'mpcy', 'potential', 'region', 'country', 'cityname', '12region']
df = pd.read_csv(mcy_pop_water_path, delimiter='|', header=None, names=column_names)
df.head(5)

,index,flag,rate,wup,pop,ava,mpcy,potential,region,country,cityname,12region
0,1,False,0.863325,36859.626,31821850.0,1.424091e+10,447.519821,93.0,eastern asia,Japan,Tokyo,East Asia
1,2,False,1.003201,21987.895,22058268.0,4.420798e+09,200.414547,39.0,southern asia,India,Delhi,South Asia
2,3,False,0.715760,20314.309,14540163.0,1.643563e+10,1130.360505,17.0,eastern asia,China,Shanghai,East Asia
3,4,False,1.002057,20136.681,20178100.0,3.006577e+08,14.900200,43.0,central america,Mexico,Ciudad de México (Mexico City),Central America
4,5,False,0.492121,19659.808,9675000.0,2.057520e+09,212.663566,11.0,south america,Brazil,São Paulo,South America


In [13]:
####################################################################################
# key_index(すべての流域を)すべてを探索する
# result_dictのavaの中に1流域からの想定取水量を入力する
# result_dictを作成する
####################################################################################
def make_result_dict(new_basin_to_cities, key_index=1):
    # unique_cties_basin: uniqueなすべての都市
    # uup: 各都市の上流にあるuniqueな都市
    edges, riv_path_array, coords_a, rivnum_list_removed = updown(new_basin_to_cities, key_index=key_index)
    unique_cities_basin = np.unique(edges)
    
    result_dict = {}
    for city_number in unique_cities_basin:
        cities_with_upstream, cities_without_upstream, uup = find_upstream_cities(edges, city_number)
        if cities_with_upstream:
            # [discharge, consumption, flag, uppoer cities]
            result_dict[city_number] = [1, 0.1, True, uup]
        elif not cities_with_upstream and cities_without_upstream:
            result_dict[city_number] = [1, 0.1, False, uup]
        else:
            result_dict[city_number] = [1, 0.1, 'Done', uup]
    
    # 同じ流域に含まれる各都市のwater availability, upstream citiesを保存している．cities_with_upstreamが存在する場合，flag=Trueとなる
    return result_dict
    
####################################################################################
# result_dictのavaの中に1流域からの想定取水量を入力する
# 一つの流域から二つ取水点がある可能性がある→どうする？
####################################################################################
def discharge_rate(city_num, riv_num, distance=100):
    city_num = int(city_num)
    riv_num = int(float(riv_num))
    print(f'city_num, riv_num, {city_num}, {riv_num}')
    
    h08dir = '/mnt/c/Users/tsimk/Downloads/dotfiles/h08'
    msk_dir = f'{h08dir}/global_city/dat/vld_cty_'
    ctymsk_path = f'{msk_dir}/city_{city_num:08}.gl5'
    ctymsk = np.fromfile(ctymsk_path, dtype='float32').reshape(2160, 4320)
    
    prf_dir = f'{h08dir}/global_city/dat/cty_prf_'
    prf_path = f'{prf_dir}/vld_cty_/city_{city_num:08}.gl5'
    prf = np.fromfile(prf_path, dtype='float32').reshape(2160, 4320)
    
    int_dir = f'{h08dir}/global_city/dat/cty_int_/{distance}km_samebasin'
    int_path = f'{int_dir}/city_{city_num:08}.gl5'
    intake = np.fromfile(int_path, dtype='float32').reshape(2160, 4320)
    # don't think about IBT updown at current stage
    intake = np.zeros((2160, 4320))

    compound = prf + intake
    rivnum_mask = np.ma.masked_where(compound == 0, rivnum_gl5)
    rivara_mask = np.ma.masked_where(compound == 0, rivara_gl5) # 複数ある
    rate = np.sum(rivara_gl5[rivnum_mask== riv_num])/np.sum(rivara_mask)
    discharge = np.sum(rivout_gl5[(compound!=0) & (rivnum_gl5==riv_num)])* 60 * 60 * 24 * 365 / (1000) # m3/year
    return rate, discharge
    
####################################################################################
# JOB
####################################################################################
key_index = 1
####################################################################################
#
new_basin_to_cities = load_json() # keys = all basins with multiple cities 180?
rate_cities = []
#
# result_dictが更新されないことが大事
result_dict =  make_result_dict(new_basin_to_cities, key_index=key_index) # keys = all cities in key_index basin
#
riv_num = list(new_basin_to_cities.keys())[key_index]
city_num_lst = list(result_dict.keys())

for city_num in city_num_lst:
    #
    rate, discharge = discharge_rate(city_num, riv_num)
    print(f'rate {rate}')
    if rate < 1:
        rate_cities.append(city_num)
    print(f'rate, discharge {rate}, {discharge}')
    #
    filtered_df = df[df['index'] == city_num]
    if filtered_df.empty:
        print(f'City number {city_num} not found in the data.')
    #
    mpcy = filtered_df.reset_index(drop=True).loc[0, 'mpcy']
    est_pop = filtered_df.reset_index(drop=True).loc[0, 'pop']
    #
    # take all available water
    if mpcy <= 1000:
        print(f'shortage')
        consumption = discharge
    #
    # take population * ratio * 1000m3/year
    else:
        print(f'no shortage')
        consumption = 1000*est_pop*rate
        if consumption < discharge:
            consumption = consumption
        else:
            consumption = discharge
    #
    print(f'consumption: {consumption}')
    result_dict[city_num][0] = discharge
    result_dict[city_num][1] = consumption
    print(f'--------------------------------------')

uid: 14.0
cities in uid
[2, 11, 126, 130, 133, 188, 197, 217, 243, 319, 321, 363, 398, 406, 455, 463, 473, 513, 526, 588, 667, 669, 673, 677, 797, 874, 883, 903, 941, 983, 1012, 1018, 1067, 1089, 1181, 1216, 1253, 1288, 1298, 1337, 1365, 1371, 1422, 1578, 1620, 1652, 1656, 1697, 1712, 1718, 1824]
city_num, riv_num, 2, 14
rate 1.0
rate, discharge 1.0, 4420797783.75
shortage
consumption: 4420797783.75
--------------------------------------
city_num, riv_num, 126, 14
rate 1.0
rate, discharge 1.0, 89442971.40234375
shortage
consumption: 89442971.40234375
--------------------------------------
city_num, riv_num, 130, 14
rate 1.0
rate, discharge 1.0, 87145943796.0
no shortage
consumption: 2960523500.0
--------------------------------------
city_num, riv_num, 133, 14
rate 1.0
rate, discharge 1.0, 6701539941.0
no shortage
consumption: 2875327750.0
--------------------------------------
city_num, riv_num, 188, 14
rate 1.0
rate, discharge 1.0, 56511942.2578125
shortage
consumption: 56511942.2578

In [46]:
# key_index=1を探索する
####################################################################################
# 上流と下流の取水の関係をrecursiveに探索
# result_dictのavaの部分を更新してreturn
# 二つの都市で，互いが互いの都市の更新を待っている状態だと，whileが無限になる
####################################################################################

def explore(result_dict):
    while True:
        all_done = True  # 全てのflagが'Done'であるかをチェックするためのフラグ
        
        for key in result_dict:
            value = result_dict[key]
            ava = value[0]
            con = value[1]
            flag = value[2]
            uup = value[3]
            
            ava_mod = ava
            
            if flag != 'Done':
                all_done = False  # まだ'Done'でないflagがある場合、all_doneをFalseに設定
        
            if flag is False:
                flg_lst = []
                for kkk in uup:
                    oth_flg = result_dict[kkk][2]
                    flg_lst.append(oth_flg)
                    
                for kkk in uup:
                    oth_con = result_dict[kkk][1]
                    ava_mod = ava_mod - np.abs(oth_con)
                result_dict[key][0] = ava_mod
                
                if con > ava_mod:
                    result_dict[key][1] = ava_mod
                    
                result_dict[key][2] = 'Done'
                
            elif flag is True:
                flg_lst = []
                for kkk in uup:
                    oth_flg = result_dict[kkk][2]
                    flg_lst.append(oth_flg)
                
                if all(item == 'Done' for item in flg_lst):
                    # 重複を削除する
                    all_upstream = []
                    for kkk in uup:
                        all_upstream.extend(result_dict[kkk][2])
                    mod_uup = [city for city in uup if city not in all_upstream]
                    for kkk in mod_uup:
                        oth_con = result_dict[kkk][1]
                        ava_mod = ava_mod - np.abs(oth_con)
                    result_dict[key][0] = ava_mod

                    if con > ava_mod:
                        result_dict[key][1] = ava_mod
                        
                    result_dict[key][2] = 'Done'
    
            else:
                flg_lst = []
                for kkk in uup:
                    oth_flg = result_dict[kkk][2]
                    flg_lst.append(oth_flg)
            print(key, flag, ava, con, flg_lst, uup)
                    
        print('------------------------------------------------------------------------------------------------------------------------------------------------------------------------')
        if all_done:
            print('all_done')
            break  # 全てのflagが'Done'であればループを終了
    return result_dict
    
result_dict_mod = explore(result_dict)
print(f'**********rivnum{list(new_basin_to_cities.keys())[key_index]}: calculation done***************')
print(f'--------------------------------------')

# uupがおかしい(find upstream)
# intakeを有効にすると↓
# 2[1089]
# 1089[1620]
# 1620[[1089]
# 2都市間で同じ流域に二つずつ取水点があり，それぞれで上下流の関係が逆になっている

1 False 11001185775.0 11001185775.0 ['Done'] [1283]
1283 Done 4749225021.0 445353750.0 [] []
------------------------------------------------------------------------------------------------------------------------------------------------------------------------
1 Done 10555832025.0 10555832025.0 ['Done'] [1283]
1283 Done 4749225021.0 445353750.0 [] []
------------------------------------------------------------------------------------------------------------------------------------------------------------------------
all_done
**********rivnum848.0: calculation done***************
--------------------------------------


In [47]:
####################################################################################
# explore all key_index & make df_new
# it takes soooo long time
# df_newを作ってdfのavailabilityの部分を更新する
distance = 100
####################################################################################

df_new = df.copy()
duplicates_citynum = [3, 35, 52, 73, 77, 85, 97, 108, 128, 129, 167, 246, 335, 345, 
                      350, 486, 490, 502, 508, 543, 550, 627, 654, 658, 764, 790, 
                      1018, 1044, 1045, 1098, 1110, 1169, 1202, 1248, 1280, 1416, 
                      1450, 1718]
duplicates_dict = {}

new_basin_to_cities = load_json() # keys = all basins with multiple cities 180?
for key_index in range(len(new_basin_to_cities.keys())):
    #
    # result_dictが更新されないことが大事
    result_dict =  make_result_dict(new_basin_to_cities, key_index=key_index) # keys = all cities in key_index basin
    #
    riv_num = list(new_basin_to_cities.keys())[key_index]
    city_num_lst = list(result_dict.keys())
    
    for city_num in city_num_lst:
        #
        rate, discharge = discharge_rate(city_num, riv_num)
        print(f'rate, discharge {rate}, {discharge/1e9} billion m3/year')
        #
        filtered_df = df[df['index'] == city_num]
        if filtered_df.empty:
            print(f'City number {city_num} not found in the data.')
        #
        mpcy = filtered_df.reset_index(drop=True).loc[0, 'mpcy']
        est_pop = filtered_df.reset_index(drop=True).loc[0, 'pop']
        #
        # take all available water
        if mpcy <= 1000:
            print(f'shortage')
            consumption = discharge
        #
        # take population * ratio * 1000m3/year
        else:
            print(f'no shortage')
            consumption = 1000*est_pop*rate
            if consumption < discharge:
                consumption = consumption
            else:
                consumption = discharge
        #
        print(f'consumption: {consumption/1e9} billion m3/year')
        result_dict[city_num][0] = discharge
        result_dict[city_num][1] = consumption
        print(f'--------------------------------------')
        
    print(f'**********rivnum{list(new_basin_to_cities.keys())[key_index]}: calculation done***************')
    print(f'--------------------------------------')

    # explore
    result_dict_mod = explore(result_dict)
    
    for city_num in result_dict_mod:
        city_num = int(city_num)
        riv_num = int(float(riv_num))
        
        h08dir = '/mnt/c/Users/tsimk/Downloads/dotfiles/h08'
        msk_dir = f'{h08dir}/global_city/dat/vld_cty_'
        ctymsk_path = f'{msk_dir}/city_{city_num:08}.gl5'
        ctymsk = np.fromfile(ctymsk_path, dtype='float32').reshape(2160, 4320)
        
        prf_dir = f'{h08dir}/global_city/dat/cty_prf_'
        prf_path = f'{prf_dir}/vld_cty_/city_{city_num:08}.gl5'
        prf = np.fromfile(prf_path, dtype='float32').reshape(2160, 4320)
        
        int_dir = f'{h08dir}/global_city/dat/cty_int_/{distance}km_samebasin'
        int_path = f'{int_dir}/city_{city_num:08}.gl5'
        intake = np.fromfile(int_path, dtype='float32').reshape(2160, 4320)
        ibt = np.sum(rivout_gl5[intake==1])*60*60*24*365/1000 # m3/year
        
        dis_els = np.sum(rivout_gl5[(prf==1) & (rivnum_gl5!=riv_num)])* 60 * 60 * 24 * 365 / (1000) # m3/year
        dis_mod = result_dict_mod[city_num][0]
        if dis_els:
            availability = dis_els + dis_mod
        else:
            availability = dis_mod
        print(f'dis_els:{dis_els}, dis_mod:{dis_mod}, availability:{availability/1e9} billion m3/year')
        print(f"mpcy: {availability/(df_new.loc[df_new['index'] == city_num, 'pop'])}")
        print(f'ava_int: {(availability+ibt)/1e9} billion m3/year')
        print(f"mpcy_int: {(availability+ibt)/df_new.loc[df_new['index'] == city_num, 'pop']}")
        print(f'--------------------------------------')
        
        df_new.loc[df_new['index'] == city_num, 'ava'] = availability
        df_new.loc[df_new['index'] == city_num, 'mpcy'] = availability/df_new.loc[df_new['index'] == city_num, 'pop']
        df_new.loc[df_new['index'] == city_num, 'ava_int'] = availability + ibt
        df_new.loc[df_new['index'] == city_num, 'mpcy_int'] = (availability+ibt)/df_new.loc[df_new['index'] == city_num, 'pop']
        
    df_new.head(10)

uid: 848.0
cities in uid
[1, 936, 1283]
city_num, riv_num, 1, 848


KeyboardInterrupt: 

In [43]:
def int_update(df_new, distance=100):
    nan_indexes = df_new[df_new['ava_int'].isna()]['index'].tolist()
    
    h08dir = '/mnt/c/Users/tsimk/Downloads/dotfiles/h08'
    rivout_path = f'{h08dir}/wsi/dat/riv_out_/W5E5LR__00000000.gl5'
    overlap_path = f'{h08dir}/global_city/dat/cty_lst_/gpw4/overlap_hidden_only.txt'
    prf_dir = f'{h08dir}/global_city/dat/cty_prf_'
    int_dir = f'{h08dir}/global_city/dat/cty_int_/{distance}km_samebasin'
    
    rivout_gl5 = np.fromfile(rivout_path, dtype='float32').reshape(2160, 4320)
    
    with open(overlap_path, 'r') as f:
        numbers = [int(line.strip()) for line in f]
    
    nan_list_removed = [num for num in nan_indexes if num not in numbers]
    nan_list_removed = [int(i) for i in nan_list_removed]

    for city_num in nan_list_removed:
    
        prf_path = f'{prf_dir}/vld_cty_/city_{city_num:08}.gl5'
        int_path = f'{int_dir}/city_{city_num:08}.gl5'
    
        if not os.path.exists(prf_path):
            print(f'{city_num} is invalid prf')
    
        else:
            intake = np.fromfile(int_path, dtype='float32').reshape(2160, 4320)
            ibt = np.sum(rivout_gl5[intake==1])*60*60*24*365/1000 # m3/year

            if not df_new[df_new['index'] == str(city_num)].empty:
                
                availability = float(df_new.loc[df_new['index'] == str(city_num), 'ava'])
                ava_int = availability + ibt
                mpcy_int = (availability+ibt)/float(df_new.loc[df_new['index'] == str(city_num), 'pop'])
                df_new.loc[df_new['index'] == str(city_num), 'ava_int'] = ava_int
                df_new.loc[df_new['index'] == str(city_num), 'mpcy_int'] = mpcy_int
                print(city_num, availability/1e9, ava_int/1e9, mpcy_int)

            else:
                print(f"No matching index found for city_num: {city_num}")
    
    return df_new

df_new = int_update(df_new)
df_new.head(10)

152 is invalid prf
168 is invalid prf
174 is invalid prf
225 is invalid prf
237 is invalid prf
273 is invalid prf
278 is invalid prf
285 is invalid prf
323 is invalid prf
393 is invalid prf
409 is invalid prf
412 is invalid prf
415 is invalid prf
422 is invalid prf
436 is invalid prf
449 is invalid prf
465 is invalid prf
471 is invalid prf
474 is invalid prf
494 is invalid prf
500 is invalid prf
528 is invalid prf
538 is invalid prf
544 is invalid prf
551 is invalid prf
552 is invalid prf
555 is invalid prf
562 is invalid prf
565 is invalid prf
572 is invalid prf
574 is invalid prf
581 is invalid prf
604 is invalid prf
614 is invalid prf
623 is invalid prf
636 is invalid prf
638 is invalid prf
675 is invalid prf
682 is invalid prf
684 is invalid prf
687 is invalid prf
694 0.0257334991875 0.6452256976875 946.536737259231
695 0.116272000125 1.792798194375 2637.725216265768
698 1.509782797125 19.676838664125 28886.393094228097
702 is invalid prf
704 is invalid prf
705 0.3705973981875 1.38

,index,flag,rate,wup,pop,ava,mpcy,potential,region,country,cityname,12region,ava_int,mpcy_int
0,1,False,0.8633253630951112,36859.626,31821850.0,13795554377.25,433.52458695047585,93.0,eastern asia,Japan,Tokyo,East Asia,17091316447.875,537.0937405548389
1,2,False,1.0032005337482284,21987.895,22058268.0,3580540135.234375,162.32190737887376,39.0,southern asia,India,Delhi,South Asia,22829217730.234375,1034.9506012999016
2,3,False,0.715759664776193,20314.309,14540163.0,13694695860.777102,941.8529806562074,17.0,eastern asia,China,Shanghai,East Asia,13926454710.097414,957.7922001354052
3,4,False,1.0020568930897795,20136.681,20178100.0,300657725.859375,14.90020001186311,43.0,central america,Mexico,Ciudad de México (Mexico City),Central America,2428205879.109375,120.338678
4,5,False,0.4921207775782957,19659.808,9675000.0,2057520001.5,212.66356604651162,11.0,south america,Brazil,São Paulo,South America,3775628136.375,390.2458022093023
5,6,False,0.4892318742858954,19312.646,9448362.0,12720310699.5,1346.2979826026988,24.0,eastern asia,Japan,Kinki M.M.A. (Osaka),East Asia,15150352164.75,1603.489808
6,7,False,0.4709559275549676,18365.262,8649229.0,632926103.34375,73.1771702823165,17.0,northern america,United States of America,New York-Newark,North America,25415389211.34375,2938.457198
7,8,False,0.7832002814418451,18257.413,14299211.0,9254410677.0,647.1972948017901,12.0,southern asia,India,Mumbai (Bombay),South Asia,15056622245.25,1052.968744
8,9,False,1.006426350884948,16899.015,17007614.0,705930828950.2577,41506.752737347975,24.0,northern africa,Egypt,Al-Qahirah (Cairo),Northern Africa,705948440199.0458,41507.78822938043
9,10,False,0.8239520931860906,16441.252,13546804.0,1971521745.2578125,145.5340865090993,24.0,eastern asia,China,Beijing,East Asia,3857525573.1328125,284.7553986263338


In [48]:
######################################################
# 更新したupdown_pathをpandas dfとして保存
#####################################################
h08dir = '/mnt/c/Users/tsimk/Downloads/dotfiles/h08'
updown_path = f'{h08dir}/wsi/dat/availablewater/updown_availability_100km.txt'
#df_new.to_csv(updown_path, sep='|', index=False)

In [49]:
######################################################
# 更新した pandas df_newを読み込み
#####################################################
import os
import json
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import pandas as pd
import matplotlib.colors as mcolors

h08dir = '/mnt/c/Users/tsimk/Downloads/dotfiles/h08'
updown_path = f'{h08dir}/wsi/dat/availablewater/updown_availability_100km.txt'
df_new = pd.read_csv(updown_path, delimiter='|', header=None)
df_new.columns = df_new.iloc[0]
df_new = df_new[1:].reset_index(drop=True)
df_new = df_new.infer_objects()
df_new.head(60)

,index,flag,rate,wup,pop,ava,mpcy,potential,region,country,cityname,12region,ava_int,mpcy_int
0,1,False,0.8633253630951112,36859.626,31821850.0,13795554377.25,433.52458695047585,93.0,eastern asia,Japan,Tokyo,East Asia,17091316447.875,537.0937405548389
1,2,False,1.0032005337482284,21987.895,22058268.0,3580540135.234375,162.32190737887376,39.0,southern asia,India,Delhi,South Asia,22829217730.234375,1034.9506012999016
2,3,False,0.715759664776193,20314.309,14540163.0,13694695860.777102,941.8529806562074,17.0,eastern asia,China,Shanghai,East Asia,13926454710.097414,957.7922001354052
3,4,False,1.0020568930897795,20136.681,20178100.0,300657725.859375,14.90020001186311,43.0,central america,Mexico,Ciudad de México (Mexico City),Central America,2428205879.109375,120.33867802763268
4,5,False,0.4921207775782957,19659.808,9675000.0,2057520001.5,212.66356604651162,11.0,south america,Brazil,São Paulo,South America,3775628136.375,390.2458022093023
5,6,False,0.4892318742858954,19312.646,9448362.0,12720310699.5,1346.2979826026988,24.0,eastern asia,Japan,Kinki M.M.A. (Osaka),East Asia,15150352164.75,1603.4898075190176
6,7,False,0.4709559275549676,18365.262,8649229.0,632926103.34375,73.1771702823165,17.0,northern america,United States of America,New York-Newark,North America,25415389211.34375,2938.4571979009634
7,8,False,0.7832002814418451,18257.413,14299211.0,9254410677.0,647.1972948017901,12.0,southern asia,India,Mumbai (Bombay),South Asia,15056622245.25,1052.9687438873375
8,9,False,1.006426350884948,16899.015,17007614.0,705930828950.2577,41506.752737347975,24.0,northern africa,Egypt,Al-Qahirah (Cairo),Northern Africa,705948440199.0458,41507.78822938043
9,10,False,0.8239520931860906,16441.252,13546804.0,1971521745.2578125,145.5340865090993,24.0,eastern asia,China,Beijing,East Asia,3857525573.1328125,284.7553986263338


In [50]:
############################################################
# バグがないか確認する(availabilityがnegativeになっていないか)
# global_city/dat/cty_lst_/gpw4/classification.txtをチェック
# NoMASKの部分がNaNになっている
###########################################################
df_new['ava'] = pd.to_numeric(df_new['ava'], errors='coerce')

# NaNが含まれていないか確認
if df_new['ava'].isna().any():
    print("The 'ava' column contains non-numeric(NaN) values.")
    non_numeric_values = df_new[pd.to_numeric(df_new['ava'], errors='coerce').isna()]['ava']
    print("Non-numeric values in 'ava' column:", non_numeric_values)
else:
    if (df_new['ava'] >= 0).all():
        print("The 'ava' column contains only non-negative values.")
    else:
        print("The 'ava' column contains negative values.")
        negative_ava_rows = df_new[df_new['ava'] < 0]
        print(negative_ava_rows)

The 'ava' column contains non-numeric(NaN) values.
Non-numeric values in 'ava' column: 151    NaN
167    NaN
173    NaN
224    NaN
236    NaN
        ..
1850   NaN
1854   NaN
1857   NaN
1858   NaN
1859   NaN
Name: ava, Length: 307, dtype: float64


In [33]:
######################################################
# updownで再計算された都市の数と流域の数を確認
# duplicatesは複数の[上下流に都市が存在する流域]に属する都市
#####################################################
new_basin_to_cities = load_json() # keys = all basins with multiple cities 186?
rivnum_lst = list(new_basin_to_cities.keys())
rivnum_lst = [int(float(i)) for i in rivnum_lst]
print(rivnum_lst)
print(f'len(rivnum_lst): {len(rivnum_lst)}')
print('----------------------------------------------------------------------------------------------------------------------------------')

cities_lst = []
for key_index, key in enumerate(new_basin_to_cities.keys()):
    key_city_list = new_basin_to_cities[key]
    key_city_list = [int(i) for i in key_city_list]

    # remove overlap
    overlap_path = f'{h08dir}/global_city/dat/cty_lst_/gpw4/overlap_hidden_only.txt'
    with open(overlap_path, 'r') as f:
        numbers = [int(line.strip()) for line in f]
    key_city_list_removed = [num for num in key_city_list if num not in numbers]
    key_city_list_removed = [int(i) for i in key_city_list_removed]
    
    cities_lst.extend(key_city_list_removed)
    #print(f'{key_index}: rivnum{key} is done')

print('----------------------------------------------------------------------------------------------------------------------------------')
print(cities_lst)
print(f'len(cities_lst): {len(cities_lst)}')
print('----------------------------------------------------------------------------------------------------------------------------------')

urivnum = np.unique(rivnum_lst)
uctynum, counts = np.unique(cities_lst, return_counts=True)
print(f'len(urivnum): {len(urivnum)}')
print(f'len(uctynum): {len(uctynum)}')
print('----------------------------------------------------------------------------------------------------------------------------------')

duplicates = uctynum[counts > 1]
print(f'duplicates cities {duplicates}')

[848, 14, 480, 5438, 164, 5, 4, 172, 11240, 189, 7763, 3, 11, 191, 138, 382, 376, 413, 2, 17, 23, 63, 182, 970, 88, 325, 955, 64, 494, 177, 343, 1830, 27, 28, 498, 528, 34, 326, 12, 1220, 1749, 228, 3829, 112, 21, 43, 1014, 618, 234, 38, 3429, 878, 82, 35, 473, 6026, 102, 581, 220, 8210, 56, 203, 243, 2370, 9930, 2360, 7178, 45, 37, 2411, 127, 805, 748, 6, 911, 53, 591, 1130, 48, 6057, 282, 6278, 1744, 1298, 25, 10, 32, 416, 1850, 1, 158, 42, 15, 244, 75, 18, 49, 146, 26, 1252, 8, 111, 40, 5127, 338, 475, 576, 696, 143, 314, 374, 74, 667, 5507, 16, 7, 46, 17041, 105, 5046, 670, 385, 166, 1207, 3477, 55, 2152, 657, 478, 10510, 2130, 358, 2753, 394, 3700, 577, 917, 3947, 78, 176, 2087, 1481, 229, 3265, 126, 586, 108, 913, 3464, 10401, 1833, 167, 323, 2715, 1882, 425, 5365, 261, 948, 284, 118, 409, 946, 1752, 359, 273, 393, 123, 231, 750, 1067, 2954, 3102, 206, 807, 65, 317, 33, 197, 1800, 508, 3374, 644, 47, 1720, 107]
len(rivnum_lst): 186
------------------------------------------------

In [46]:
df_new['index'] = df_new['index'].astype(str)
duplicates_lst = list(map(str, duplicates))
mpcy_values = df_new[df_new['index'].isin(duplicates_lst)]['mpcy']
print(mpcy_values)

2        941.8529506168535
34       7.769037635829043
51        8096.91174532348
72        8432.50937000074
76      10.302880930542878
84      19.346183389005166
96       5234.646262909766
107     1604.8806390091847
127     14.365305329978847
128      302.4333636297578
166      8.688201096150761
245      70.31971922390305
334     217.31024336360318
344     35.986495601745546
349      56.33714634774938
485     43326.187551640374
489     10007.036641406761
501     1289938.2943092918
507     165.30080826404907
542      9910.084089415996
549      4418.347533398172
626     146.59960476002828
653     180826.46942545325
657     3969.1503013629385
763      93572.85228983799
789     1782111.0994622197
1017     537.0966117262542
1043    371.61615266221577
1044    119.55885896817638
1097    138582.20254251905
1109    107.84520933231602
1168    1684.7750612598338
1201    201.07126985469168
1247      94.1711118669255
1279    3731.4939874818156
1415     885.6684651875955
1449     4965.736581533075
1